In [1]:
#impot libary
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.model_selection import GridSearchCV


In [2]:
#import kaggle system for upload test,train,submission data
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [3]:
#read train data
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
#read test data
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
#read submission data
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")



In [4]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [6]:
train.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

In [7]:
train['Cabin']

0          B/0/P
1          F/0/S
2          A/0/S
3          A/0/S
4          F/1/S
          ...   
8688      A/98/P
8689    G/1499/S
8690    G/1500/S
8691     E/608/S
8692     E/608/S
Name: Cabin, Length: 8693, dtype: object

In [8]:
cat_cols = ["HomePlanet","CryoSleep","Cabin","Destination", "VIP"]
cont_cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [10]:
simp_num = SimpleImputer(missing_values = np.nan, strategy = "mean")
simp_cat = SimpleImputer(missing_values = np.nan, strategy = "most_frequent")

In [11]:
for col in cat_cols:
    train[col] = simp_cat.fit_transform(train[col].values.reshape(-1,1))[:,0]
    test[col] = simp_cat.transform(test[col].values.reshape(-1,1))[:,0]

for col in cont_cols:
    train[col] = simp_num.fit_transform(train[col].values.reshape(-1,1))[:,0]
    test[col] = simp_num.transform(test[col].values.reshape(-1,1))[:,0]

In [12]:
# New features - training set
train['Deck'] = train['Cabin'].apply(lambda x: str(x).split('/')[0])
train['Number'] = train['Cabin'].apply(lambda x:  str(x).split('/')[1]).astype(int)
train['Side'] = train['Cabin'].apply(lambda x:  str(x).split('/')[2])

# New features - test set
test['Deck'] = test['Cabin'].apply(lambda x: str(x).split('/')[0])
test['Number'] = test['Cabin'].apply(lambda x: str(x).split('/')[1]).astype(int)
test['Side'] = test['Cabin'].apply(lambda x: str(x).split('/')[2])

In [13]:
cat_cols.append('Deck')
cat_cols.append('Side')
cont_cols.append('Number')

In [14]:
def encoder(df):
    
    for col in cat_cols:
        le=LabelEncoder()
        le.fit(df[col])
        df[col]=le.transform(df[col])
        
    for col in cont_cols:
        df[col]=np.log1p(df[col])
    
    return df


In [15]:
target = train['Transported'] 
let = LabelEncoder()
let.fit(target)
train = encoder(train)
test = encoder(test)

In [16]:
train.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               81
VIP                2
RoomService     1274
FoodCourt       1508
ShoppingMall    1116
Spa             1328
VRDeck          1307
Name            8473
Transported        2
Deck               8
Number          1817
Side               2
dtype: int64

In [17]:
y = let.transform(target)


cols_to_drop = ['PassengerId', 'Cabin', 'Name','Transported']
cols_to_drop_test = ['PassengerId', 'Cabin', 'Name']

train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop_test, axis=1)

X = train.values

In [18]:
classifier = KNeighborsClassifier(n_neighbors = 7, weights = 'uniform',metric = 'manhattan')
classifier.fit(X, y)

y_pred = classifier.predict(test.values)

In [19]:
y_pred

array([0, 0, 1, ..., 1, 1, 1])

In [20]:
y_pred = y_pred.astype(str)
y_pred = np.char.replace(y_pred, '0', 'False')
y_pred = np.char.replace(y_pred, '1', 'True') 
y_pred

array(['False', 'False', 'True', ..., 'True', 'True', 'True'], dtype='<U5')

In [21]:
submission['Transported'] = y_pred
submission.to_csv('submission.csv', index=False)
my_submission = pd.read_csv("submission.csv")
my_submission

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True
